<a href="https://colab.research.google.com/github/New-Masster/Assistente-de-Voz-Multi-Idiomas-Com-Whisper-e-Gemini/blob/main/Assistente_de_Voz_Multi_Idiomas_Com_Whisper_e_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
language = 'pt'

# 1. Gravação de Áudio Com Python (e Uma Pitada de JavaScript) 🎤

In [2]:
# Referência: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be

from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

# Código JavaScript para gravar áudio do usuário usando a "MediaStream Recording API"
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  # Executa o código JavaScript para gravar o áudio
  display(Javascript(RECORD))
  # Recebe o áudio gravado como resultado do JavaScript
  js_result = output.eval_js('record(%s)' % (sec * 1000))
   # Decodifica o áudio em base64
  audio = b64decode(js_result.split(',')[1])
  # Salva o áudio em um arquivo
  file_name = 'request_audio.wav'
  with open(file_name, 'wb') as f:
    f.write(audio)
  # Retorna o caminho do arquivo de áudio (pasta padrão do Google Colab)
  return f'/content/{file_name}'

# Grava o áudio do usuário por um tempo determinado (padrão 5 segundos)
print('Ouvindo...\n')
record_file = record()
print('Gravação concluída!\n')

# Exibe o áudio gravado
display(Audio(record_file, autoplay=False))

Ouvindo...



<IPython.core.display.Javascript object>

Gravação concluída!



# 2. Reconhecimento de Fala com Whisper (OpenAI) 🧠

In [3]:
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import whisper

# Selecione o modelo do Whisper que melhor atenda às suas necessidades:
# https://github.com/openai/whisper#available-models-and-languages
model = whisper.load_model("small")

# Transcreve o audio gravado anteriormente.
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]
print(transcription)

 Responda com uma frase, o que é Python?


# 3. Integração com a API do Gemini 💬

In [10]:
import google.generativeai as genai
from google.colab import userdata
import os


# No Colab, adicionei a chave ao gerenciador de segredos em "🔑" no painel esquerdo,
# assim posso compartilhar este projeto sem medo de compartilhar minha chave API também.
# Dei a ela o nome `GOOGLE_API_KEY`.
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)


In [6]:
import google.generativeai as genai
from IPython.display import Markdown, display

# 1. Definição do Modelo:
# Usei o 'gemini-flash-latest'
MODEL_ID = 'models/gemini-flash-latest'

def processar_transcricao(texto_transcrito):
    """
    Envia o texto para a IA, formata a visualização e retorna o conteúdo.
    """
    # Validação: Evita gastar tokens da API se o áudio não gerou texto.
    if not texto_transcrito.strip():
        print("⚠️ Erro: A variável 'transcription' está vazia.")
        return None

    try:
        # 2. Inicialização do Modelo com o SDK do Google.
        model = genai.GenerativeModel(MODEL_ID)

        # 3. Chamada da API: Transforma sua transcrição em uma resposta inteligente.
        response = model.generate_content(texto_transcrito)

        # 4. Formatação: O Markdown deixa títulos e listas legíveis no Colab.
        print("✅ Processamento concluído com sucesso!\n")
        display(Markdown(f"### Resposta do Gemini:\n{response.text}"))

        # 5. Saída de Dados: O 'return' permite que o texto saia da função
        # e vá para a variável global 'gemini_response'.
        return response.text

    except Exception as e:
        # Tratamento: Captura erros de rede ou limites de cota.
        print(f"❌ Ocorreu um erro técnico: {e}")
        return None

# 6. EXECUÇÃO E CAPTURA:
# Atribuíndo o resultado à variável 'gemini_response'.
gemini_response = processar_transcricao(transcription);

✅ Processamento concluído com sucesso!



### Resposta do Gemini:
Python é uma linguagem de programação de alto nível, interpretada e versátil, amplamente reconhecida por sua sintaxe clara e facilidade de leitura.

# 4. Sintetizando a Resposta do Gemini Como Voz (gTTS) 🔊

In [7]:
!pip install gTTS

In [11]:
from gtts import gTTS
from IPython.display import Audio, display

# Cria um objeto gTTS com a resposta gerada pelo Gemini e a língua que será sintetizada em voz (variável "language").
texto_para_voz = gemini_response
gtts_object = gTTS(text=texto_para_voz, lang='pt', slow=False)

# Salva o áudio da resposta no arquivo especificado (pasta padrão do Google Colab)
response_audio = "/content/response_audio.wav"
gtts_object.save(response_audio)

# Reproduz o áudio da resposta salvo no arquivo
display(Audio(response_audio, autoplay=True))